In [45]:
from glob import glob
import os
import subprocess
import nibabel as nib
import numpy as np
import pandas as pd
import utils
import scipy.ndimage
import scipy
import torch

from skimage.measure import regionprops

In [167]:
def getRegionsAsCSV(mask):    
    img = nib.load(mask).get_fdata().astype(np.uint8)
    regions = regionprops(img)
    df = pd.DataFrame()
    for props in regions:
        x,y,z = props.centroid
        x = x #+ 1 # +1 because ITK-Snap is between 1-128 and Python 0-127
        y = y #+ 1
        z = z #+ 1
        t = 0
        label = props.label
        mass = props.area
        volume = props.area
        count = len(props.coords)
        row = {
            'x': x,
            'y': y,
            'z': z,
            't': t,
            'label': label,
            'mass': mass,
            'volume': volume,
            'count': count,
        }
        temp_df = pd.DataFrame(data=row, index=[0])
        df = pd.concat([df, temp_df], ignore_index=True)

    return df

In [168]:
# Masks Labels to CSV
for dataset in ["Feminad", "Painfact", "IRIS"]:
    print(dataset)
    data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset2/" + str(dataset) + "/")
    masks = sorted(glob(os.path.join(data_dir, 'Landmarks_Mask_Resample', "*.nii.gz")))
    print(len(masks))
    for i, mask in enumerate(masks):
        outname = data_dir + "Landmarks_CSV_Resample/" + mask.split('/')[-1].split('.')[0] + ".csv"
        regions_df = getRegionsAsCSV(mask)
        regions_df.to_csv(outname, index=False)
    print('-'*40)
    break

Feminad
5
----------------------------------------


In [169]:
# Template Labels to CSV
template = os.path.join("/home/valentini/dev/Mousenet/dataset2/Atlas/", "Deformable_Feminad_Landmarks_template0_sameaffine.nii.gz")
template_out = os.path.join("/home/valentini/dev/Mousenet/dataset2/Atlas/", "Deformable_Feminad_Landmarks_template0_sameaffine.csv")
regions_df = getRegionsAsCSV(template)
regions_df.to_csv(template_out, index=False)

In [170]:
def interpolate_warp(warp_data, point):
    x_range = np.arange(0, warp_data.shape[1], 1)
    y_range = np.arange(0, warp_data.shape[2], 1)
    z_range = np.arange(0, warp_data.shape[3], 1) 
    
    x = point[0]
    y = point[1]
    z = point[2]
    
    interp_x = scipy.interpolate.RegularGridInterpolator((x_range, y_range, z_range), warp_data[0,:,:,:], fill_value=0)
    dx = interp_x(point)
    
    interp_y = scipy.interpolate.RegularGridInterpolator((x_range, y_range, z_range), warp_data[1,:,:,:], fill_value=0)
    dy = interp_y(point)
    
    interp_z = scipy.interpolate.RegularGridInterpolator((x_range, y_range, z_range), warp_data[2,:,:,:], fill_value=0)
    dz = interp_z(point)
    
    
    return dx, dy, dz

In [185]:
def applyWarpToCSV(csv, warp, physical=True):    
    if '.mat' in warp:
        warp = utils.get_warp_from_mat_file("dataset2/Atlas/Deformable_Feminad_template0_sameaffine.nii.gz", warp)         
    else:
        warp = nib.load(warp) 
        
        
    if physical:
        warp_voxel = utils.convert_warp_physical_to_voxel(warp)
    else:
        warp_voxel = warp
        
    #print(warp.shape)
    #print(warp.get_fdata()[53, 44, 101, 0,:])
    #print(warp_voxel.shape)
    #print(warp_voxel.get_fdata()[0, :, 53, 44, 101])
    
    df = pd.read_csv(csv)
    warp_data = warp_voxel.get_fdata()
    if not physical:
        warp_data = torch.from_numpy(warp_data).permute(3,4,0,1,2).numpy()
        
    new_df = pd.DataFrame()
    for i in range(len(df)): 
        x = df['x'][i]# - 1 # -1 because ITK-Snap is between 1-128 and Python 0-127
        y = df['y'][i]# - 1
        z = df['z'][i]# - 1
        print(warp_data.shape)
        dx, dy, dz = interpolate_warp(warp_data[0,:,:,:,:], (x,y,z))
            
        
        if physical:
            new_x = x - dx #+ 1 # +1 because ITK-Snap is between 1-128 and Python 0-127
            new_y = y - dy #+ 1
            new_z = z - dz #+ 1
        else:
            new_x = x + dx #+ 1 # +1 because ITK-Snap is between 1-128 and Python 0-127
            new_y = y + dy #+ 1
            new_z = z + dz #+ 1
            
        new_t = df['t'][i]
        new_label = df['label'][i]
        new_mass = df['mass'][i]
        new_volume = df['volume'][i]
        new_count = df['count'][i]
        
        row = {
            'x': new_x,
            'y': new_y,
            'z': new_z,
            't': new_t,
            'label': new_label,
            'mass': new_mass,
            'volume': new_volume,
            'count': new_count,
        }
        temp_df = pd.DataFrame(data=row, index=[0])
        new_df = pd.concat([new_df, temp_df], ignore_index=True)
        
    return new_df

In [177]:
# MasksCSVs to AffineCSV + DeformableCSV (with ANTS)
for dataset in ["Feminad", "Painfact", "IRIS"]:
    print(dataset)
    data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset2/" + str(dataset) + "/")
    csvs = sorted(glob(os.path.join(data_dir, 'Landmarks_CSV_Resample', "*.csv")))
    warps = sorted(glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Affine_Template', "*.mat")))
    for i, csv in enumerate(csvs):
        print(str(i+1) + "/" + str(len(csvs)), end="\r")
        outname = data_dir + "Landmarks_CSV_Resample_Affine/Affine_" + csvs[i].split('/')[-1].split('.')[0] + ".csv"
        name = csvs[i].split('/')[-1].split('.')[0].split('_')
        name = name[-2] + '_' + name[-1]
        warp = glob(os.path.join(data_dir, 'MRI_N4_Resample_Norm_Deformable', "*_" + str(name) + "*.mat"))[0]  
        regions_df = applyWarpToCSV(csvs[i], warp, physical=True)
        regions_df.to_csv(outname, index=False)
        
        outname2 = data_dir + "Landmarks_CSV_Resample_Deformable/Deformable_" + csvs[i].split('/')[-1].split('.')[0] + ".csv"
        warp2 = warp.split('GenericAffine')[0][:-1] + "1Warp.nii.gz"        
        regions_df_2 = applyWarpToCSV(outname, warp2, physical=True)
        regions_df_2.to_csv(outname2, index=False)
    print('-'*40)
    break

Feminad
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1

In [186]:
# MasksCSVs to AffineCSV + DeformableCSV (with DEEP LEARNING)
for output_folder in ["DL_Localnet", 
                      "DL_Localnet_Painfact", 
                      "DL_Localnet_Reg"]:
    dataset = "Feminad"
    print(dataset)
    data_dir = os.path.join("/home/valentini/dev/Mousenet/dataset2/" + str(dataset) + "/")
    warps_dir = os.path.join("/home/valentini/dev/Mousenet/output/" + str(dataset) + "/" + str(output_folder) + "/")
    csvs = sorted(glob(os.path.join(data_dir, 'Landmarks_CSV_Resample_Affine', "*.csv")))
    for i, csv in enumerate(csvs):
        print(str(i+1) + "/" + str(len(csvs)), end="\r")
        outname = warps_dir + "/Landmarks_Deformable/Deformable_" + csvs[i].split('/')[-1].split('.')[0] + ".csv"
        name = csvs[i].split('/')[-1].split('.')[0].split('_')
        name = name[-2] + '_' + name[-1]        
        warp = glob(os.path.join(warps_dir, 'DeformableWarp', "*" + str(name) + "*.nii.gz"))[0]  
        print(outname)
        print(warp)
        regions_df = applyWarpToCSV(csvs[i], warp, physical=False)
        regions_df.to_csv(outname, index=False)
    print('-'*40)

Feminad
/home/valentini/dev/Mousenet/output/Feminad/DL_Localnet//Landmarks_Deformable/Deformable_Affine_Landmarks_2_6516.csv
/home/valentini/dev/Mousenet/output/Feminad/DL_Localnet/DeformableWarp/Affine_template0Norm_Resampled_N4_2_651613WarpedToTemplate.nii.gz
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
/home/valentini/dev/Mousenet/output/Feminad/DL_Localnet//Landmarks_Deformable/Deformable_Affine_Landmarks_2_6517.csv
/home/valentini/dev/Mousenet/output/Feminad/DL_Localnet/DeformableWarp/Affine_template0Norm_Resampled_N4_2_651714WarpedToTemplate.nii.gz
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
(1, 3, 128, 128, 128)
/home/valentini/dev/Mousenet/output/Feminad/DL_Localnet//Landmarks_Deformable/Deformable_Affine_Landmarks_2_6518.csv
/home/valentini/dev/Mousenet/output/Feminad/DL_Localnet/DeformableWarp/Affine_template0Norm_Resampled_N

In [187]:
def compute_csv_distance(csv1, csv2):
    df1 = pd.read_csv(csv1, sep=',', header=0)
    df2 = pd.read_csv(csv2, sep=',', header=0)
    TREs = []
    for i in df1['label']:
        ind1 = np.where(df1['label'] == i)[0]
        ind2 = np.where(df2['label'] == i)[0]
        if len(ind1) == 1 and len(ind2) == 1:
            x1 = df1['x'][ind1[0]]
            x2 = df2['x'][ind2[0]]            
            y1 = df1['y'][ind1[0]]
            y2 = df2['y'][ind2[0]]            
            z1 = df1['z'][ind1[0]]
            z2 = df2['z'][ind2[0]]  
            
            TRE = np.sqrt((x1-x2)*(x1-x2)+(y1-y2)*(y1-y2)+(z1-z2)*(z1-z2))
            TREs.append(TRE)    
    return TREs

In [190]:
# Compute CSV distance between Template and Registered MRIs DL / between template and Registered MRIs ANTS
template_csv = "dataset2/Atlas/Deformable_Feminad_Landmarks_template0_sameaffine.csv"

ants_def_dir = os.path.join("dataset2", "Feminad")
ants_def_csvs = sorted(glob(os.path.join(ants_dir, 'Landmarks_CSV_Resample_Deformable', "*.csv")))

ants_aff_dir = os.path.join("dataset2", "Feminad")
ants_aff_csvs = sorted(glob(os.path.join(ants_dir, 'Landmarks_CSV_Resample_Affine', "*.csv")))

dl_dir = os.path.join("output", "Feminad", "DL_Localnet")
dl_csvs = sorted(glob(os.path.join(dl_dir, 'Landmarks_Deformable', "*.csv")))

dl2_dir = os.path.join("output", "Feminad", "DL_Localnet_Painfact")
dl2_csvs = sorted(glob(os.path.join(dl2_dir, 'Landmarks_Deformable', "*.csv")))

dl3_dir = os.path.join("output", "Feminad", "DL_Localnet_Reg")
dl3_csvs = sorted(glob(os.path.join(dl3_dir, 'Landmarks_Deformable', "*.csv")))

if(len(ants_csvs) == len(dl_csvs)):
    for i in range(len(ants_csvs)):        
        print(ants_csvs[i])
        ants_aff_distances = compute_csv_distance(ants_aff_csvs[i], template_csv)
        ants_def_distances = compute_csv_distance(ants_def_csvs[i], template_csv)
        dl_distances = compute_csv_distance(dl_csvs[i], template_csv)
        dl2_distances = compute_csv_distance(dl2_csvs[i], template_csv)
        dl3_distances = compute_csv_distance(dl3_csvs[i], template_csv)
        
        ants_aff_distance = np.mean(ants_aff_distances)        
        ants_def_distance = np.mean(ants_def_distances)
        dl_distance = np.mean(dl_distances)
        dl2_distance = np.mean(dl2_distances)
        dl3_distance = np.mean(dl3_distances)
        
        print(ants_aff_distances)
        print(ants_def_distances)
        print(dl_distances)
        print(dl2_distances)
        print(dl3_distances)
        print('-'*20)
        
        print(ants_aff_distance)
        print(ants_def_distance)
        print(dl_distance)
        print(dl2_distance)
        print(dl3_distance)
        
        print('-'*50)


dataset2/Feminad/Landmarks_CSV_Resample_Deformable/Deformable_Landmarks_2_6516.csv
[1.615925465053312, 1.4499763269809087, 3.6322596495344195, 2.232630903615903, 2.827085469715069, 1.0191319265006855]
[2.4751415272923736, 0.6790571739148029, 4.314134034404713, 2.541478811572919, 3.0370132448158356, 1.5659647643134447]
[2.3430499493189143, 0.8767856968899717, 2.8147161576622737, 0.696665857217919, 2.340854649075076, 1.1748753367570741]
[0.9811567159414442, 0.3310521381852397, 3.105235339795599, 1.289955511635348, 1.9772032718183197, 0.9436064189273136]
[1.891811706410343, 0.781346811989492, 2.945979704505958, 1.1083308534374474, 2.0048890800673984, 1.2677593326989989]
--------------------
2.1295016235667164
2.4354649260523478
1.707824607820205
1.4380348993838774
1.6666862481849396
--------------------------------------------------
dataset2/Feminad/Landmarks_CSV_Resample_Deformable/Deformable_Landmarks_2_6517.csv
[1.0845398148036354, 0.11021153242886288, 1.590082994809218, 2.244343517966